In [27]:
from dataset1 import *
import numpy as np
from scipy.linalg import solve_triangular
from numpy.linalg import eigh
import torch
from hadamard_transform import hadamard_transform
import time
#test

## Define functions

In [100]:
def generate_gaussian_matrix(seed, shape):
    """Generate a Gaussian random matrix with the specified seed and shape."""
    rng = np.random.default_rng(seed)
    return rng.normal(size=shape)

def getError(A,B):
    return np.linalg.norm(A-B,ord='nuc')/np.linalg.norm(A,ord='nuc')

def fjlt(A, D, indices, direction):
    """
    Applies the Fast Johnson-Lindenstrauss Transform (FJLT) to a square matrix.

    Parameters:
    - A: Input square matrix (NumPy array of size n x m).
    - D: Diagonal vector of +1 and -1 (NumPy array of size n).
    - indices: Indices of the selected rows or columns for subsampling (NumPy array of size l).
    - direction: 'left' to apply FJLT from the left (on rows) or 'right' for the transpose case (on columns).

    Returns:
    - A_proj: Transformed matrix after applying FJLT and subsampling.
    """
    n, m = A.shape
    l = indices.shape[0]  # Sketch size
    if direction == 'left':  # FJLT applied to the rows of A
        assert len(D) == n, "The length of D must match the number of rows in A."
        assert (n & (n - 1) == 0), "The number of rows in A must be a power of 2."
        # Multiply each row of A by the corresponding element in D
        A_tilde = A * D[:, np.newaxis]
        # Apply the Fast Hadamard Transform to the rows
        A_fht = hadamard_transform(torch.tensor(A_tilde.T)).numpy().T
        # Subsample the rows and scale
        A_proj = A_fht[indices, :] * np.sqrt(n / l)

    elif direction == 'right':  # FJLT applied to the columns of A
        # Multiply each column of A by the corresponding element in D
        A_tilde = A * D[np.newaxis, :]
        # Apply the Fast Hadamard Transform to the columns
        A_fht = hadamard_transform(torch.tensor(A_tilde)).numpy()
        # Subsample the columns and scale
        A_proj = A_fht[:, indices] * np.sqrt(n / l)
    
    return A_proj

def getLowRank(A, rank ,sketch_dim,sketch_method='gaussian',good_conditionned=True): 
    """
    Compute a low-rank approximation of the matrix A using 
    Eigenvalue decomposition and truncation.

    Parameters:
        A (numpy.ndarray): Original matrix.
        Omega (numpy.ndarray): Random sketching matrix.
        rank (int): Desired rank for the approximation.

    Returns:
        A_nyst_k (numpy.ndarray): Low-rank approximation of A.
    """
    m = A.shape[0]
    B = None
    C = None

    if sketch_method == 'fjlt': #If not gaussian sketching use Fast Johnson-Lindenstrauss transform to sketch
        D = np.random.choice([1, -1], size=m)
        indices = np.random.choice(m, size=sketch_dim, replace=False) # Subsample
        C = fjlt(A,D,indices,direction='right')
        B = fjlt(C,D,indices,direction='left')
        
    elif sketch_method == 'gaussian': # Use gaussian matrix to sketch
        
        #Omega = np.random.normal(size=(m,sketch_dim)) #Sketch matrix
        base_seed_sequence = np.random.SeedSequence(12345)
        # Generate sqrt_p independent subsequences
        sub_sequences = base_seed_sequence.spawn(2)
        seed_values = [int(seq.generate_state(1)[0]) for seq in sub_sequences]
        omega1 = generate_gaussian_matrix(seed_values[0],(int(m/2),sketch_dim))
        omega2 = generate_gaussian_matrix(seed_values[1],(int(m/2),sketch_dim))
        Omega = np.concatenate((omega1,omega2))
        # Compute sketch matrix C
        C = A @ Omega  # Project A onto the subspace defined by Omega
        # Compute the smaller B matrix
        B = Omega.T @ C  # Compression of A into a smaller matrix via Omega
    else:
        return None
    
    if good_conditionned:
        # Perform Cholesky decomposition of B
        L = np.linalg.cholesky(B)  # L is a lower triangular matrix
        # Solve for Z using the triangular solver
        Z = solve_triangular(L, C.T, lower=True).T  # Z = C @ L^-T --> ZL^T  = C -- > LZ^T = C^T

    else: #if not good conditionned
        # Eigenvalue decomposition of B
        eig_v, V = eigh(B)  # eig_v: Eigenvalues, V: Eigenvectors

        # Truncate eigenvalues and eigenvectors to keep only positive eigenvalues
        truncate = eig_v > 0
        eig_v = eig_v[truncate]
        V = V[:, truncate]
        # Normalize using the eigenvalues to construct Z
        Z = C @ (V * (1 / np.sqrt(eig_v) ) ) # Normalize columns of C

    # Perform QR decomposition of Z
    Q, R = np.linalg.qr(Z)  # Q: Orthogonal basis, R: Upper triangular matrix
    # Singular Value Decomposition (SVD) of R
    U, Sigma, Vh = np.linalg.svd(R)

    # Construct the low-rank approximation
    U_hat = Q @ U[:, :rank]  # Truncate U to the desired rank
    A_nyst_k = (U_hat * (Sigma[:rank]**2)) @ U_hat.T  # Final approximation
    return A_nyst_k


In [ ]:

def getLowRank_timed(A, rank, sketch_dim, sketch_method='gaussian', good_conditionned=True):
    """
    Compute a low-rank approximation of the matrix A using sketching, 
    eigenvalue decomposition, and truncation. Prints the time taken for each major operation.

    Parameters:
        A (numpy.ndarray): Original matrix (size m x n).
        rank (int): Desired rank for the approximation.
        sketch_dim (int): Dimension of the sketching subspace.
        sketch_method (str): Method used for sketching. Options are:
                             - 'gaussian': Uses a Gaussian random matrix for sketching.
                             - 'fjlt': Uses the Fast Johnson-Lindenstrauss Transform.
        good_conditionned (bool): If True, uses Cholesky decomposition for better conditioning. 
                                  Otherwise, uses eigenvalue decomposition.

    Returns:
        A_nyst_k (numpy.ndarray): Low-rank approximation of A (size m x m).
    """
    m = A.shape[0]
    B = None
    C = None

    start_time = time.time()

    # Apply sketching based on the selected method
    if sketch_method == 'fjlt':  # Use Fast Johnson-Lindenstrauss Transform for sketching
        D = np.random.choice([1, -1], size=m)
        indices = np.random.choice(m, size=sketch_dim, replace=False)
        sketch_start = time.time()
        C = fjlt(A, D, indices, direction='right')
        B = fjlt(C, D, indices, direction='left')
        print(f"FJLT sketching completed in {time.time() - sketch_start:.6f} seconds.")

    elif sketch_method == 'gaussian':  # Use Gaussian random matrix for sketching
        Omega = np.random.normal(size=(m, sketch_dim))
        sketch_start = time.time()
        C = A @ Omega
        B = Omega.T @ C
        print(f"Gaussian sketching completed in {time.time() - sketch_start:.6f} seconds.")
    else:
        return None

    decomposition_start = time.time()

    if good_conditionned:
        # Use Cholesky decomposition
        cholesky_start = time.time()
        L = np.linalg.cholesky(B)
        print(f"Cholesky decomposition completed in {time.time() - cholesky_start:.6f} seconds.")
        triangular_start = time.time()
        Z = solve_triangular(L, C.T, lower=True).T
        print(f"Triangular solve completed in {time.time() - triangular_start:.6f} seconds.")
    else:
        # Use eigenvalue decomposition
        eigh_start = time.time()
        eig_v, V = eigh(B)
        print(f"Eigenvalue decomposition completed in {time.time() - eigh_start:.6f} seconds.")
        truncate_start = time.time()
        truncate = eig_v > 0
        eig_v = eig_v[truncate]
        V = V[:, truncate]
        Z = C @ (V * (1 / np.sqrt(eig_v)))
        print(f"Eigenvalue truncation and normalization completed in {time.time() - truncate_start:.6f} seconds.")

    qr_start = time.time()
    Q, R = np.linalg.qr(Z)
    print(f"QR decomposition completed in {time.time() - qr_start:.6f} seconds.")

    svd_start = time.time()
    U, Sigma, Vh = np.linalg.svd(R)
    print(f"SVD completed in {time.time() - svd_start:.6f} seconds.")

    approx_start = time.time()
    U_hat = Q @ U[:, :rank]
    A_nyst_k = (U_hat * Sigma[:rank]**2) @ U_hat.T
    print(f"Low-rank approximation construction completed in {time.time() - approx_start:.6f} seconds.")

    print(f"Total computation time: {time.time() - start_time:.6f} seconds.")
    return A_nyst_k


## Tests

In [96]:
n = 1024
A = getExpMatrix(n,R=2)


In [104]:
A_lowrank = getLowRank_timed(A,rank=64,sketch_dim=128,sketch_method='gaussian',good_conditionned=False)
print(getError(A,A_lowrank))

Gaussian sketching completed in 0.002101 seconds.
Eigenvalue decomposition completed in 0.000903 seconds.


ValueError: operands could not be broadcast together with shapes (128,95) (95,95) 

In [25]:
A_lowrank = getLowRank_timed(A,rank=32,sketch_dim=128,sketch_method='gaussian',good_conditionned=False)
print(getError(A,A_lowrank))

Gaussian sketching completed in 0.032449 seconds.
Eigenvalue decomposition completed in 0.000000 seconds.
Eigenvalue truncation and normalization completed in 0.015323 seconds.
QR decomposition completed in 0.009999 seconds.
SVD completed in 0.001001 seconds.
Low-rank approximation construction completed in 0.018281 seconds.
Total computation time: 0.084976 seconds.
3.6005374612291685e-07
